In [ ]:
from proximal_operators_and_gradients import (
    gen_matrices, proj_linf_annulus, proj_simplex, proj_l1_ball)
from numpy.linalg import norm, inv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Grad simplex

In [ ]:
def proj_grad_dirty(mat, vec, err_max, ite_max):
    dim = len(mat)
    sqr = np.matmul(mat.T, mat)
    eta = 1 / norm(sqr)  # mat is not zero
    mul_mat = np.eye(dim) - eta * sqr
    out = eta * np.matmul(mat.T, vec)
    prim = np.zeros(dim)
    err = err_max + 1
    ite = 0
    errs = list()
    while err > err_max and ite < ite_max:
        prim_new = proj_simplex(np.matmul(mul_mat, prim) + out, 1)
        err = norm(prim - prim_new)
        errs.append([err, norm(mat @ prim_new - vec)])
        prim = prim_new.copy()
        ite += 1
    return prim, errs

In [ ]:
mat, vec = gen_matrices(1000)[0:2]
res, errs = proj_grad_dirty(mat, vec, 1e-5, 10000)
errs = np.array(errs)
errs[:, 1] = errs[:, 1] - errs[-1, 1]
plt.plot(np.log(errs))
res.sum()

In [ ]:
def proj_grad_simplex(mat, vec, rad=1, err_max=1e-5, ite_max=1000):
    dim = len(mat)
    sqr = np.matmul(mat.T, mat)
    eta = 1 / norm(sqr)  # mat is not zero
    mul_mat = np.eye(dim) - eta * sqr
    out = eta * np.matmul(mat.T, vec)
    prim = np.zeros(dim)
    err = err_max + 1
    ite = 0
    errs = list()
    while err > err_max and ite < ite_max:
        prim_new = proj_simplex(np.matmul(mul_mat, prim) + out, rad)
        err = norm(prim - prim_new)
        errs.append(err)
        prim = prim_new.copy()
        ite += 1
    return prim, errs

In [ ]:
%%timeit
mat, vec = gen_matrices(1000)[0:2]
res, errs = proj_grad_simplex(mat, vec)

# Grad admm

In [ ]:
def grad_admm_dirty(mat, vec, rad_1, rad_infty, err_max, ite_max):
    dim = len(mat)
    sqr = np.matmul(mat.T, mat)
    eta = 1 / norm(sqr)  # mat is not zero
    mul_mat = np.eye(dim) - eta * sqr
    out = eta * np.matmul(mat.T, vec)
    concensus = np.zeros(dim)
    dual_1 = np.zeros(dim)
    dual_2 = np.zeros(dim)
#     v_min = -rad_infty
#     v_max = +rad_infty
    err = err_max + 1
    ite = 0
    errs = list()
    while err > err_max and ite < ite_max:
        prim = np.matmul(mul_mat, concensus) + out
        prim_1 = proj_linf_annulus(
            concensus - dual_1, -rad_infty, +rad_infty)
        prim_2 = proj_l1_ball(concensus - dual_2, rad_1)
        concensus_new = 1 / 3 * (prim + prim_1 + prim_2)
        dual_1_new = dual_1 + prim_1 - concensus
        dual_2_new = dual_2 + prim_2 - concensus
        err_prim = [
            norm(concensus_new - prim),
            norm(concensus_new - prim_1),
            norm(concensus_new - prim_2),
        ]
        err_dual = [
            norm(concensus - concensus_new),
            norm(prim_1 - concensus),
            norm(prim_2 - concensus)]
        err = max(err_prim + err_dual)
        errs.append(err_prim + err_dual + [norm(mat @ concensus - vec)])
        concensus = concensus_new.copy()
        dual_1 = dual_1_new.copy()
        dual_2 = dual_2_new.copy()
        ite += 1
    return concensus, errs

In [ ]:
mat, vec = gen_matrices(1000)[0:2]
res, errs = grad_admm_dirty(mat, vec, 1, 0.005, 1e-5, 10000)
errs = np.array(errs)
errs[:, -1] = errs[:, -1] - errs[-1, -1]
pd.DataFrame(errs).apply(np.log).plot()
# plt.plot(np.log(errs))
np.abs(res).max(), np.abs(res).sum()  # , res

# Concensus ADMM

In [ ]:
def concensus_admm_dirty(
    mat, vec, rad_1, rad_infty, err_max=1e-5, ite_max=1000):
    rho = 0.001
    dim = len(mat)
    sqr = np.matmul(mat.T, mat)
    mul_mat = inv(np.eye(dim) + rho * sqr)
    out = rho * np.matmul(mul_mat, np.matmul(mat.T, vec))
    concensus = np.zeros(dim)
    dual = np.zeros(dim)
    dual_1 = np.zeros(dim)
    dual_2 = np.zeros(dim)
#     v_min = -rad_infty
#     v_max = +rad_infty
    err = err_max + 1
    ite = 0
    errs = list()
    while err > err_max and ite < ite_max:
        prim = np.matmul(mul_mat, concensus - dual) + out
        prim_1 = proj_linf_annulus(
            concensus - dual_1, -rad_infty, +rad_infty)
        prim_2 = proj_l1_ball(concensus - dual_2, rad_1)
        concensus_new = 1 / 3 * (prim + prim_1 + prim_2)
        dual_new = dual + prim - concensus
        dual_1_new = dual_1 + prim_1 - concensus
        dual_2_new = dual_2 + prim_2 - concensus
        err_prim = [
            norm(concensus_new - prim),
            norm(concensus_new - prim_1),
            norm(concensus_new - prim_2),
        ]
        err_dual = [
            rho * norm(concensus - concensus_new),
            rho * norm(prim_1 - concensus),
            rho * norm(prim_2 - concensus)]
        err = max(err_prim + err_dual)
        errs.append(err_prim + err_dual + [norm(mat @ concensus - vec)])
        concensus = concensus_new.copy()
        dual = dual_new.copy()
        dual_1 = dual_1_new.copy()
        dual_2 = dual_2_new.copy()
        ite += 1
    return concensus, errs

In [ ]:
mat, vec = gen_matrices(1000)[0:2]
res, errs = concensus_admm_dirty(mat, vec, 1, 0.005, 1e-5, 10000)
errs = np.array(errs)
errs[:, -1] = errs[:, -1] - errs[-1, -1]
pd.DataFrame(errs).apply(np.log).plot()
# plt.plot(np.log(errs))
np.abs(res).max(), np.abs(res).sum()  # , res

In [ ]:
%%timeit
mat, vec = gen_matrices(1000)[0:2]
res, errs = concensus_admm_dirty(mat, vec, 1, 0.005)